In [1]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake (and underactuated).
if 'google.colab' in sys.modules and importlib.util.find_spec('underactuated') is None:
    urlretrieve(f"http://underactuated.csail.mit.edu/scripts/setup/setup_underactuated_colab.py",
                "setup_underactuated_colab.py")
    from setup_underactuated_colab import setup_underactuated
    setup_underactuated(underactuated_sha='15cfd96b0bdfd1b0c67597c24f91907776c02a6d', drake_version='0.27.0', drake_build='release')

server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']
# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc_planar, zmq_url_planar, web_url_planar = start_zmq_server_as_subprocess(server_args=server_args)
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

import numpy as np
from ipywidgets import FloatSlider, ToggleButton
from IPython.display import display, SVG, HTML
import pydot

import pydrake.all
from pydrake.all import (
    MultibodyPlant, DiagramBuilder, LinearQuadraticRegulator, Saturation, SceneGraph, Simulator, 
    WrapToSystem, AddMultibodyPlantSceneGraph, Parser, MathematicalProgram, eq, PiecewisePolynomial,
    SnoptSolver, TrajectorySource, MultibodyPositionToGeometryPose, PlanarSceneGraphVisualizer
)
from pydrake.examples.acrobot import AcrobotPlant, AcrobotGeometry
from pydrake.systems.jupyter_widgets import WidgetSystem
from pydrake.common.containers import namedview
from underactuated import FindResource
from underactuated.jupyter import running_as_notebook


Cloning into '/opt/underactuated'...

HEAD is now at 15cfd96 and again

ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
ERROR: bokeh 2.3.2 has requirement pillow>=7.1.0, but you'll have pillow 7.0.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.






## Problem Description

We want to write a nonlinear optimization for the ball balancing bot. In order to complete our trajectory optimization, we will use `MathematicalProgram`.

**maybe write more later if we submit the notebook

## Parse the `urdf` and Get the `MultibodyPlant`

We start by defining a couple of physical parameters that we will need below.

In [2]:
# friction coefficient between feet and ground
friction = .2

# position of the feet in the respective leg frame
# (must match the urdf)
foot_in_leg = {
    'stance_leg': np.zeros(3),        # stance foot in stance-leg frame
    'swing_leg': np.array([0, 0, -1]) # swing foot in swing-leg frame
}

In [3]:
ballbot_urdf = """
<?xml version="1.0"?>

<robot xmlns="http://drake.mit.edu"
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 name="BallBot">

  <link name="ground">
    <visual>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
      <material>
        <color rgba="0.93 .74 .4 1" />
      </material>
    </visual>
  </link>

  <joint name="ground_weld" type="fixed">
    <parent link="world" />
    <child link="ground" />
  </joint>

  <link name="ball">
    <inertial>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <mass value="5" />
      <inertia ixx=".02" ixy="0" ixz="0" iyy="0.02" iyz="0" izz="0.02" />
    </inertial>
    <visual>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <geometry>
        <sphere radius=".1" />
        <box size="0.1 0.1 0.1"/>
      </geometry>
      <material>
        <color rgba="0.25 0.52 0.96 1" />
      </material>
    </visual>
  </link>

  <link name="bot">
    <inertial>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <mass value="4" />
      <inertia ixx="0.018" ixy="0" ixz="0" iyy="0.018" iyz="0" izz="0.0288" />
    </inertial>
    <visual>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <geometry>
         <cylinder length=".1" radius=".12" />
      </geometry>
      <material>
        <color rgba=".61 .63 .67 1" />
      </material>
    </visual>
  </link>
  
  <joint name="x" type="prismatic">
    <parent link="world" />
    <child link="ball" />
    <origin xyz="0 0 .1" />
    <axis xyz="1 0 0" />
    <dynamics damping="0.1" />
  </joint>

  <joint name="theta" type="continuous">
    <parent link="ball" />
    <child link="bot" />
    <origin xyz="0 0 0" />
    <axis xyz="0 1 0" />
    <dynamics damping="0.1" />
  </joint>

  <transmission type="SimpleTransmission" name="ball_torque">
    <actuator name="torque" />
    <joint name="theta" />
    <mechanicalReduction>1</mechanicalReduction>
  </transmission>

  <transmission type="SimpleTransmission" name="ball_force">
    <actuator name="force" />
    <joint name="x" />
    <mechanicalReduction>.1</mechanicalReduction>
  </transmission>
</robot>
"""

In [4]:
# parse urdf and create the MultibodyPlant
ballbot_2D = MultibodyPlant(time_step=0)
Parser(ballbot_2D).AddModelFromString(ballbot_urdf, "urdf")
ballbot_2D.Finalize()

# overwrite MultibodyPlant with its autodiff copy
ballbot_2D = ballbot_2D.ToAutoDiffXd() 

# number of configuration variables
nq = ballbot_2D.num_positions()  # Returns the size of the multibody system state vector x = [q v]. This will be num_positions() plus num_velocities()

# number of components of the contact forces
nf = 2

In [5]:
print(nq)

2


## Helper Functions

In [42]:
def ballbot2D_dynamics(vars):
    split_at = [nq, 2 * nq, 3 * nq]
    q, qd, qdd, u = np.split(vars, split_at)
    
    # set compass gait state
    context = ballbot_2D.CreateDefaultContext()
    ballbot_2D.SetPositions(context, q)
    ballbot_2D.SetVelocities(context, qd)
    # ballbot_2D.get_actuation_input_port().FixValue(context, [u,u])
    
    # matrices for the manipulator equations
    M = ballbot_2D.CalcMassMatrixViaInverseDynamics(context)
    Cv = ballbot_2D.CalcBiasTerm(context)
    tauG = ballbot_2D.CalcGravityGeneralizedForces(context)
    print(M.dot(qdd) + Cv - tauG)
    
    # return violation of the manipulator equations
    return M.dot(qdd) + Cv - tauG

## The Trajectory Optimization Problem

We start by setting some parameters of our optimization problem.

In [54]:
# time steps in the trajectory optimization
T = 50

# minimum and maximum time interval is seconds
h_min = .005
h_max = .05

# goal, start positions
goal = [0, 0]
start = [.7, .7]

In [55]:
# initialize program
prog = MathematicalProgram()

# vector of the time intervals
# (distances between the T + 1 break points)
h = prog.NewContinuousVariables(T, name='h')

# system configuration, generalized velocities, and accelerations
q = prog.NewContinuousVariables(rows=T+1, cols=nq, name='q')
qd = prog.NewContinuousVariables(rows=T+1, cols=nq, name='qd')
qdd = prog.NewContinuousVariables(rows=T, cols=nq, name='qdd')

# actuation
u = prog.NewContinuousVariables(rows=T, cols=nq, name='u')

Here are part of the constraints of the optimization problem:


1.   Time steps need to be bounded between h_min and h_max
2.   Map configurations, velocities, accelerations to each other using the Euler method
3.   Something about dynamics
4.   Want the thing on top to be horizonal at the end (theta1 = 0, theta2 = 0)
5.   Can add an angle bound (like it can't be more than 15 deg away from horizontal)



In [56]:
# 1. lower and upper bound on the time steps for all t
prog.AddBoundingBoxConstraint([h_min] * T, [h_max] * T, h)

# 2. link the configurations, velocities, and accelerations
# uses implicit Euler method, https://en.wikipedia.org/wiki/Backward_Euler_method
for t in range(T):
    prog.AddConstraint(eq(q[t+1], q[t] + h[t] * qd[t+1]))
    prog.AddConstraint(eq(qd[t+1], qd[t] + h[t] * qdd[t]))

# 3. drake dynamics
for t in range(T):
  vars = np.concatenate((q[t], qd[t], qdd[t], u[t]))
  prog.AddConstraint(ballbot2D_dynamics, lb=[0]*nq, ub=[0]*nq, vars=vars)
  # residual = ballbot2D_dynamics(vars)
  # prog.AddLinearConstraint(residual[0] == 0)
  # prog.AddLinearConstraint(residual[1] == 0)
# vars = np.concatenate((q[-1], qd[-1], qd_post, imp))
# prog.AddConstraint(reset_velocity_heelstrike, lb=[0]*(nq+nf), ub=[0]*(nq+nf), vars=vars)    

# 4. horizontal at end
# q = [x, theta1]
prog.AddLinearConstraint(q[-1][1] == goal[1])

# at the origin at the end
prog.AddLinearConstraint(q[-1][0] == goal[0])

# give start position
prog.AddLinearConstraint(q[0][0] == start[0])
prog.AddLinearConstraint(q[0][1] == start[1])

Here we set the initial guess for our optimization problem.

For the time steps `h` we just initialize them to their maximal value `h_max` (somewhat an arbitrary decision, but it works).

For the robot configuration `q`, we interpolate between the initial value `q0_guess` and the final value `- q0_guess`.
In our implementation, the value given below for `q0_guess` made the optimization converge.
But, if you find the need, feel free to tweak this parameter.
The initial guess for the velocity and the acceleration is obtained by differentiating the one for the position.

The normal force `f` at the stance foot is equal to the total `weight` of the robot.

All the other optimization variables are initialized at zero.
(Note that, if the initial guess for a variable is not specified, the default value is zero.)

In [57]:
# vector of the initial guess
initial_guess = np.empty(prog.num_vars())

# initial guess for the time step
h_guess = h_max
prog.SetDecisionVariableValueInVector(h, [h_guess] * T, initial_guess)

# linear interpolation of the configuration
q0_guess = np.array([0, .15])
q_guess_poly = PiecewisePolynomial.FirstOrderHold(
    [0, T * h_guess],
    np.column_stack((q0_guess, - q0_guess))
)
qd_guess_poly = q_guess_poly.derivative()
qdd_guess_poly = q_guess_poly.derivative()

# set initial guess for configuration, velocity, and acceleration
q_guess = np.hstack([q_guess_poly.value(t * h_guess) for t in range(T + 1)]).T
qd_guess = np.hstack([qd_guess_poly.value(t * h_guess) for t in range(T + 1)]).T
qdd_guess = np.hstack([qdd_guess_poly.value(t * h_guess) for t in range(T)]).T
prog.SetDecisionVariableValueInVector(q, q_guess, initial_guess)
prog.SetDecisionVariableValueInVector(qd, qd_guess, initial_guess)
prog.SetDecisionVariableValueInVector(qdd, qdd_guess, initial_guess)

We can finally solve the problem! Be sure that the solver actually converged: you can check this by looking at the variable result.is_success() (printed below).

In [58]:
# solve mathematical program with initial guess
solver = SnoptSolver()
result = solver.Solve(prog, initial_guess)

# ensure solution is found
print(f'Solution found? {result.is_success()}.')

[<AutoDiffXd -0.020211559434072275 nderiv=8>
 <AutoDiffXd -1.26731510236035 nderiv=8>]
[<AutoDiffXd -0.024164885897565963 nderiv=8>
 <AutoDiffXd -0.28491259508123784 nderiv=8>]
[<AutoDiffXd -0.024168014169860433 nderiv=8>
 <AutoDiffXd -0.27325743838625877 nderiv=8>]
[<AutoDiffXd -0.024170272396254935 nderiv=8>
 <AutoDiffXd -0.26159256541264664 nderiv=8>]
[<AutoDiffXd -0.024171660495453558 nderiv=8>
 <AutoDiffXd -0.24991839609456884 nderiv=8>]
[<AutoDiffXd -0.024172178417484876 nderiv=8>
 <AutoDiffXd -0.23823535070085983 nderiv=8>]
[<AutoDiffXd -0.02417182614370376 nderiv=8>
 <AutoDiffXd -0.22654384981989206 nderiv=8>]
[<AutoDiffXd -0.02417060368679203 nderiv=8>
 <AutoDiffXd -0.21484431434443466 nderiv=8>]
[<AutoDiffXd -0.024168511090757988 nderiv=8>
 <AutoDiffXd -0.2031371654565011 nderiv=8>]
[<AutoDiffXd -0.02416554843093488 nderiv=8>
 <AutoDiffXd -0.19142282461218704 nderiv=8>]
[<AutoDiffXd -0.024161715813978135 nderiv=8>
 <AutoDiffXd -0.1797017135264977 nderiv=8>]
[<AutoDiffXd -0.02

In the following cell we retrieve the optimal value of the decision variables.

In [59]:
# get optimal solution
h_opt = result.GetSolution(h)
q_opt = result.GetSolution(q)
qd_opt = result.GetSolution(qd)
qdd_opt = result.GetSolution(qdd)

# stack states
x_opt = np.hstack((q_opt, qd_opt))

## Animate the Result

Here we quickly build a Drake diagram to animate the result we got from trajectory optimization: useful for debugging your code and to be sure that everything looks good.

In [60]:
# interpolate state values for animation
time_breaks_opt = np.array([sum(h_opt[:t]) for t in range(T+1)])
x_opt_poly = PiecewisePolynomial.FirstOrderHold(time_breaks_opt, x_opt.T)

# parse urdf with scene graph
ballbot = MultibodyPlant(time_step=0)
scene_graph = SceneGraph()
ballbot.RegisterAsSourceForSceneGraph(scene_graph)
Parser(ballbot).AddModelFromString(ballbot_urdf, "urdf")
ballbot.Finalize()

# build block diagram and drive system state with
# the trajectory from the optimization problem
builder = DiagramBuilder()
source = builder.AddSystem(TrajectorySource(x_opt_poly))
builder.AddSystem(scene_graph)
pos_to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(ballbot, input_multibody_state=True))
builder.Connect(source.get_output_port(0), pos_to_pose.get_input_port())
builder.Connect(pos_to_pose.get_output_port(), scene_graph.get_source_pose_port(ballbot.get_source_id()))

# add visualizer
xlim = [-.75, .75]
ylim = [-.2, 1.5]
visualizer = builder.AddSystem(PlanarSceneGraphVisualizer(scene_graph, xlim=xlim, ylim=ylim, show=False))
builder.Connect(scene_graph.get_pose_bundle_output_port(), visualizer.get_input_port(0))
simulator = Simulator(builder.Build())

# generate and display animation
visualizer.start_recording()
simulator.AdvanceTo(x_opt_poly.end_time())
ani = visualizer.get_recording_as_animation()
HTML(ani.to_jshtml())